## Sentiment Analysis
### Built specifically for cekmedsos.com

Storage for tweet data is stored on cekmedsos.com, communicating through database layer

In [1]:
# Imports
import pymysql.cursors
import pandas as pd
import uuid
import time
import os
import string
import re

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords 
from datetime import datetime

# database connection properties
db_host='202.157.176.225'
db_user ='cekmedsos_db'
db_password='kuku838485*#'
db_database='cekmedsos_database'

In [2]:
# Connect to the database
connection = pymysql.connect(host=db_host,
                             user=db_user,
                             password=db_password,
                             database=db_database,
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)

cursor = connection.cursor()

# get available jobs from database server, first come first serve
# sql = "select id, hastag, `parameter` from screen_analisis_ai where active = 1 and status = 1 and jenis_analisa = 1 order by created asc, id asc limit 1"
sql = "select id, hastag, `parameter` \
from screen_analisis_ai \
where active = 1 \
and status = 1 \
and jenis_analisa = 2 \
order by created asc, id asc limit 1"

row_count = cursor.execute(sql)

if(row_count == 0):
    # get out, nothing to do
    print('Zero jobs, quitting now')
    quit()

result = cursor.fetchall()
database_keyword_id = result[0]['hastag']
similarity_treshold = result[0]['parameter']
i_process_id = result[0]['id']
screen_name = ''

print(database_keyword_id)
print(similarity_treshold)
print(i_process_id)

#pigaihinasukujawa
0.85
693


## Query tweet data from database
using parameter retrieve from job list

In [3]:
# Connect to the database
connection = pymysql.connect(host='202.157.176.225',
                             user='cekmedsos_db',
                             password='kuku838485*#',
                             database='cekmedsos_database',
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)

cursor = connection.cursor()

# s_query_string = 'select id, tweet, tweet_date_time from ret_tweet where '
s_query_string = 'select id, tweet, tweet_date_time from vw_ret_tweet_clean where '

if (screen_name != ''):
    # print('use screen name')
    s_query_string = s_query_string + 'screen_name = "' + screen_name + '" and db_id = "' + str(database_keyword_id) + '"'
else:
    # print('no use')
    s_query_string = s_query_string + 'keyword = "' + database_keyword_id.replace('"','') + '" limit 100'
    
print(s_query_string)
df = pd.read_sql(s_query_string, con=connection)

# Close Connection
connection.close()

# see result
df.head()

select id, tweet, tweet_date_time from vw_ret_tweet_clean where keyword = "#pigaihinasukujawa" limit 100


,id,tweet,tweet_date_time
0,1354579947801759748,Jangan mancing di air keruh... #PigaiHinaSuk...,2021-01-28 07:00:01
1,1354609891395182592,Kritik itu baik untuk pemerintah. Kritik itu ...,2021-01-28 08:59:00
2,1354610143015673856,Natalius Pigai ngajak gelut cah... #PigaiHin...,2021-01-28 09:00:00
3,1354610143019888644,""" Natalius Pigai telah menghina suku Jawa dan ...",2021-01-28 09:00:00
4,1354610143032451072,Apa yang kalian tangkap dari pernyataan Pigai ...,2021-01-28 09:00:00


In [4]:
# function stemming
import nltk
from sklearn.pipeline import Pipeline
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

# nltk.download('stopwords')

def stemming(comment):
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    do = []
    for w in comment:
        dt = stemmer.stem(w)
        do.append(dt)
    d_clean = []
    d_clean = " ".join(do)
    return d_clean
    
# function case folding
import re
def casefolding(comment):
    comment = comment.lower()
    comment = comment.strip(" ")
    comment = re.sub(r'[?|$|.|!_:")(-+,]','',comment)
    return comment

# Text Preprocessing, 
def text_preproc(strIn):
    # case folding
    strOut = strIn.lower()
    # remove numbers
    strOut = re.sub(r"\d+", "", strOut)
    # remote punctuation
    strOut = strOut.translate(str.maketrans("","",string.punctuation))
    # remove whitspace
    strOut = strOut.strip()
    # 
    strOut = re.sub('\s+',' ',strOut)
    return strOut


## Remote stop words, stemming, and tokenizing

In [5]:
df['tweet'] = df['tweet'].apply(casefolding)
df['tweet'] = df['tweet'].apply(text_preproc)

sw = stopwords.words('indonesian')

#tokenized
df['tokenized_tweet'] = df.apply(lambda row: nltk.word_tokenize(row['tweet']), axis=1)

# apply stopword removal
df['sw'] = df.apply(lambda row: {w for w in row['tokenized_tweet'] if not w in sw}, axis=1)

df['tweet'] = df['tweet'].apply(stemming)
df.head(100)

,id,tweet,tweet_date_time,tokenized_tweet,sw
0,1354579947801759748,janganmancingdiairkeruhpigaihinasukujawahttpst...,2021-01-28 07:00:01,"[jangan, mancing, di, air, keruh, pigaihinasuk...","{pigaihinasukujawa, httpstcogooflieq, air, man..."
1,1354609891395182592,kritikitubaikuntukpemerintahkritikitumenyoroti...,2021-01-28 08:59:00,"[kritik, itu, baik, untuk, pemerintah, kritik,...","{melukai, menghina, pigaihinasukujawa, program..."
2,1354610143015673856,nataliuspigaingajakgelutcahpigaihinasukujawaup...,2021-01-28 09:00:00,"[natalius, pigai, ngajak, gelut, cah, pigaihin...","{ngajak, pigaihinasukujawa, cah, upkeun, gelut..."
3,1354610143019888644,nataliuspigaitelahmenghinasukujawadaninimemicu...,2021-01-28 09:00:00,"[natalius, pigai, telah, menghina, suku, jawa,...","{menghina, memicu, pigaihinasukujawa, httpstco..."
4,1354610143032451072,apayangkaliantangkapdaripernyataanpigaidivideo...,2021-01-28 09:00:00,"[apa, yang, kalian, tangkap, dari, pernyataan,...","{pigaihinasukujawa, pernyataan, httpstcobphizr..."
...,...,...,...,...,...
95,1354610147432095746,jangansampaikitaterprovokasidengankalimatpigai...,2021-01-28 09:00:01,"[jangan, sampai, kita, terprovokasi, dengan, k...","{httpstcobcxwatuo, pigaihinasukujawa, terprovo..."
96,1354610147520368640,pigaihinasukujawakurangasemnihorgrasisamathttp...,2021-01-28 09:00:01,"[pigaihinasukujawa, kurang, asem, nih, org, ra...","{pigaihinasukujawa, org, nih, asem, httpstcoor..."
97,1354610147595718658,abangpigaiinibternyaajagojugayamenjadiprovokat...,2021-01-28 09:00:01,"[abang, pigai, inibternyaa, jago, juga, ya, me...","{pigaihinasukujawa, bg, abang, anak, ya, pulau..."
98,1354610147658665991,gakjauhbedakalianduabgemangkalianpasanganseras...,2021-01-28 09:00:01,"[gak, jauh, beda, kalian, dua, bg, emang, kali...","{pasangan, pigaihinasukujawa, emang, bg, seras..."


## Proses TF-IDF
short for term frequency–inverse document frequency, is a numerical statistic that is intended to reflect how important a word is to a document in a collection or corpus

In [6]:
# converting


In [7]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

tf = TfidfVectorizer()
text_tf = tf.fit_transform(df['sw'].astype('U'))
text_tf

<100x543 sparse matrix of type '<class 'numpy.float64'>'
	with 1038 stored elements in Compressed Sparse Row format>

In [8]:
# Data Splitting
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(text_tf, df['sw'], test_size=0.5, random_state=42)

## Perform algo Naive-Bayes
check to https://en.wikipedia.org/wiki/Naive_Bayes_classifier


In [9]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

# changes to string t
y_train = y_train.astype('string')


clf = MultinomialNB().fit(x_train, y_train)
predicted = clf.predict(x_test)

In [10]:
print(df['sw'].astype('U'))

0     {'pigaihinasukujawa', 'httpstcogooflieq', 'air...
1     {'melukai', 'menghina', 'pigaihinasukujawa', '...
2     {'ngajak', 'pigaihinasukujawa', 'cah', 'upkeun...
3     {'menghina', 'memicu', 'pigaihinasukujawa', 'h...
4     {'pigaihinasukujawa', 'pernyataan', 'httpstcob...
                            ...                        
95    {'httpstcobcxwatuo', 'pigaihinasukujawa', 'ter...
96    {'pigaihinasukujawa', 'org', 'nih', 'asem', 'h...
97    {'pigaihinasukujawa', 'bg', 'abang', 'anak', '...
98    {'pasangan', 'pigaihinasukujawa', 'emang', 'bg...
99    {'pigaihinasukujawa', 'pernyataan', 'memiliki'...
Name: sw, Length: 100, dtype: object
